In [2]:
%pip install pandas yfinance

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.0 MB 1.1 MB/s eta 0:00:03
     ---------- ----------------------------- 0.8/3.0 MB 1.1 MB/s eta 0:00:02
     ----------------- ---------------------- 1.3/3.0 MB 1.5 MB/s eta 0:00:02
     --------------------------- ------------ 2.1/3.0 MB 2.0 MB/s eta 0:00:01
     ---------------------------------- ----- 2.6/3.0 MB 2.1 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 2.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished w

In [26]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import os

In [45]:
def get_intraday_data(ticker, period="5d", interval="5m", cache_dir="intraday_cache"):
    os.makedirs(cache_dir, exist_ok=True)
    
    filename = f"{ticker}_{period}_{interval}.csv"
    filepath = os.path.join(cache_dir, filename)

    # ✅ If cache exists and is fresh
    if os.path.exists(filepath):
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(filepath))
        if datetime.now() - file_mod_time < timedelta(hours=24):
            print(f"📁 Loading cached data from: {filepath}")
            df = pd.read_csv(filepath, index_col=0, parse_dates=True)
            return df

        else:
            print(f"🗑 Deleting outdated file: {filepath}")
            os.remove(filepath)

    # ✅ Download fresh data
    print(f"🌐 Downloading fresh data for {ticker}...")
    df = yf.download(ticker, period=period, interval=interval)

    if df.empty:
        raise ValueError("❌ Downloaded data is empty. Try again later.")

    # ✅ Flatten multi-index columns (if any)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # ✅ Save flat version
    df.to_csv(filepath)
    print(f"✅ Data saved to: {filepath}")
    return df


In [56]:
def exponential_moving_average(prices, window):
    ema = []
    multiplier = 2 / (window + 1)
    for i in range(len(prices)):
        if i < window - 1:
            ema.append(None)
        elif i == window - 1:
            sma = sum(prices[:window]) / window
            ema.append(sma)
        else:
            ema.append((prices[i] - ema[-1]) * multiplier + ema[-1])
    return ema

def backtest_ema_strategy(prices, ema_short_period=9, ema_long_period=21):
    short_ema = exponential_moving_average(prices, ema_short_period)
    long_ema = exponential_moving_average(prices, ema_long_period)

    signals = []
    position = None
    correct_predictions = 0
    total_trades = 0
    total_return = 0.0

    for i in range(1, len(prices)):
        if (short_ema[i] is None or long_ema[i] is None or
            short_ema[i-1] is None or long_ema[i-1] is None):
            signals.append(None)
            continue

        # BUY SIGNAL
        if short_ema[i-1] < long_ema[i-1] and short_ema[i] > long_ema[i]:
            if position is None:
                position = ('buy', i)
            signals.append("buy")

        # SELL SIGNAL
        elif short_ema[i-1] > long_ema[i-1] and short_ema[i] < long_ema[i]:
            if position is None:
                position = ('sell', i)
            signals.append("sell")

        else:
            signals.append(None)

        # EXIT TRADE ON OPPOSITE SIGNAL
        if position and (
            (position[0] == 'buy' and signals[-1] == "sell") or 
            (position[0] == 'sell' and signals[-1] == "buy")
        ):
            entry_type, entry_idx = position
            entry_price = prices[entry_idx]
            exit_price = prices[i]
            
            if entry_type == 'buy':
                pnl = (exit_price - entry_price) / entry_price * 100
                if exit_price > entry_price:
                    correct_predictions += 1
            else:  # short
                pnl = (entry_price - exit_price) / entry_price * 100
                if exit_price < entry_price:
                    correct_predictions += 1

            total_return += pnl
            total_trades += 1
            position = None

    accuracy = (correct_predictions / total_trades) * 100 if total_trades > 0 else 0
    return signals, accuracy, total_trades, total_return


# Load data
ticker = "AAPL"
df = get_intraday_data("AAPL", period="8d", interval="1m")
print(df.columns)

close_prices = df["Close"].tolist()

signals, accuracy, total_trades, total_return = backtest_ema_strategy(close_prices, 9, 21)

print(f"✅ Trades: {total_trades}")
print(f"🎯 Accuracy: {accuracy:.2f}%")
print(f"📈 Total Return: {total_return:.2f}%")


📁 Loading cached data from: intraday_cache\AAPL_8d_1m.csv
Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
✅ Trades: 69
🎯 Accuracy: 26.09%
📈 Total Return: 0.02%


In [ ]:
def optimize_ema_strategy(prices, short_range=(5, 15), long_range=(16, 50)):
    best_accuracy = 0
    best_total_return = -float('inf')
    best_params = None

    for short_ema in range(short_range[0], short_range[1] + 1):
        for long_ema in range(long_range[0], long_range[1] + 1):
            if short_ema >= long_ema:
                continue  # Invalid configuration

            signals, accuracy, total_trades, total_return = backtest_ema_strategy(prices, short_ema, long_ema)

            # You can choose what to maximize: accuracy or total_return
            if accuracy > best_accuracy and total_return > 0:
                best_accuracy = accuracy
                best_total_return = total_return
                best_params = (short_ema, long_ema)

    return {
        "Best Short EMA": best_params[0],
        "Best Long EMA": best_params[1],
        "Accuracy": best_accuracy,
        "Total Return (%)": best_total_return
    }
result = optimize_ema_strategy(close_prices, short_range=(5, 15), long_range=(16, 50))
print("📊 Best EMA Params & Results:")
print(result)


📊 Best EMA Params & Results:
{'Best Short EMA': 8, 'Best Long EMA': 26, 'Accuracy': 34.84848484848485, 'Total Return (%)': 1.4691270406748378}
